In [1]:
from google.colab import drive
import shutil

drive.mount('/content/drive', force_remount=True)
source_folder_path1 = '/content/drive/MyDrive/FaceAlignment'
destination_folder_path1 = '/content/FaceAlignment'
shutil.copytree(source_folder_path1, destination_folder_path1)

source_folder_path2 = '/content/drive/MyDrive/FaceAlignmentV2'
destination_folder_path2 = '/content/FaceAlignmentV2'
shutil.copytree(source_folder_path2, destination_folder_path2)

Mounted at /content/drive


'/content/FaceAlignmentV2'

In [ ]:
import glob
import zipfile
import os

for (op, model) in [(op, model) for op in ["cropped","aligned"] for model in ["mtcnn","rf"]]:
  zip_file_path = f"/content/FaceAlignmentV2/IJBB/ijbb_{model}_{op}.zip"
  extracted_folder_path = f"/content/ijbb_{model}_{op}"
  os.makedirs(extracted_folder_path, exist_ok=True)
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extracted_folder_path)

**FaceQAN Codes**


In [2]:
!git clone https://github.com/LSIbabnikz/FaceQAN.git

Cloning into 'FaceQAN'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 32 (delta 4), reused 0 (delta 0), pack-reused 16 (from 1)
Receiving objects: 100% (32/32), 23.99 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
!cp /content/FaceAlignment/FaceQAN/backbone.pth /content/FaceQAN/src/models/weights.pth

In [4]:
%cd /content/FaceQAN/src

/content/FaceQAN/src


In [ ]:
import sys
sys.path.append("./")
import os
import csv
import glob

from PIL import Image
import torch
from torchvision.transforms import transforms

from util import F, load_cosface, add_norm_to_model
from bim_attack import basic_iterative_method as BIM
from symmetry import symmetry_estimation


def __calculate_score_single__(model: torch.nn.Module, image_transform: transforms.Compose, image_path: str, eps: float, l: int, k: int, p: int) -> float:
    """Method performs FaceQAN over individual images

    Args:
        model (torch.nn.Module): FR model used for adversarial attack.
        image_transform (transforms.Compose): Image transformation, without normalization.
        image_path (str): Path to image, for which the quality score will be predicted.
        eps (float): ε-parameter from the paper.
        l (int): l-parameter from the paper, controling the number of FGSM iterations to perform.
        k (int): k-parameter from the paper, controling the number of adversarial examples generated.
        p (int): p-parameter from the paper, used in the final quality score calculation as the exponent.

    Returns:
        float: The calculated quality score
    """
    image = Image.open(image_path).convert("RGB")
    S_i = BIM(model, image_transform(image), eps=eps, iter=l)
    q_s = symmetry_estimation(model, image_transform, image)
    quality_score = F(S_i, q_s)
    return quality_score

eps,l,k,p = 0.001, 5, 10, 5
model = load_cosface().eval()
model = add_norm_to_model(model,mean=[.5, .5, .5],std=[.5, .5, .5])
image_transforms = transforms.Compose([transforms.Resize((112, 112)),transforms.ToTensor()])

op = "cropped"
detector = "rf"

csv_file = f'/content/ijbb_{detector}_{op}_faceqan.csv'
dataset_path = f'/content/ijbb_{detector}_{op}/*.jpg'

with open(csv_file, mode='w', newline='') as file:
  csv_writer = csv.writer(file)
  csv_writer.writerow(["Image","Score"])
  for index,image_path in enumerate(glob.glob(dataset_path)):
    qs = __calculate_score_single__(model,image_transforms,image_path,eps,l,k,p)
    csv_writer.writerow([image_path.split("/")[-1], qs])
    print(f'Image {index} -- {image_path} -- Quality score {qs}')


/content/FaceQAN/src/models/iresnet.py:207: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(os.path.dirname(__file__), "weights.p

Streaming output truncated to the last 5000 lines.
Image 5690 -- /content/ijbb_rf_cropped/52364.jpg -- Quality score 0.07408601610302577
Image 5691 -- /content/ijbb_rf_cropped/149841.jpg -- Quality score 0.12423271140708267
Image 5692 -- /content/ijbb_rf_cropped/71528.jpg -- Quality score 0.2553379555551158
Image 5693 -- /content/ijbb_rf_cropped/14883.jpg -- Quality score 0.2901133510973736
Image 5694 -- /content/ijbb_rf_cropped/23364.jpg -- Quality score 0.41772907307129503
Image 5695 -- /content/ijbb_rf_cropped/160211.jpg -- Quality score 0.07761699376281753
Image 5696 -- /content/ijbb_rf_cropped/96989.jpg -- Quality score 0.2553210857728297
Image 5697 -- /content/ijbb_rf_cropped/97782.jpg -- Quality score 0.10767883464450544
Image 5698 -- /content/ijbb_rf_cropped/203764.jpg -- Quality score 0.2845685043327211
Image 5699 -- /content/ijbb_rf_cropped/63387.jpg -- Quality score 0.2745973465841549
Image 5700 -- /content/ijbb_rf_cropped/11043.jpg -- Quality score 0.30156722597834207
Image

In [ ]:
import shutil

op = "cropped"
detector = "rf"

csv_file = f"/content/ijbb_{detector}_{op}_faceqan.csv"
source_file = csv_file
destination_directory = "/content/drive/MyDrive/FaceAlignmentV2/IJBB"
shutil.copy(source_file, destination_directory)
print(f"File copied from {source_file} to {destination_directory}")

File copied from /content/ijbb_rf_cropped_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/IJBB


## SCFace

In [5]:
import glob
import zipfile
import os

for d,c in [(d,c) for d in range(1,4) for c in range(1,6)]:
  for (op, model) in [(op, model) for op in ["cropped","aligned"] for model in ["mtcnn","rf"]]:
    scface = f'scface_d{d}_c{c}_{model}_{op}'
    zip_file_path = f"/content/FaceAlignmentV2/SCFace/{scface}.zip"
    extracted_folder_path = f"/content/{scface}"
    os.makedirs(extracted_folder_path, exist_ok=True)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_folder_path)

In [ ]:
import sys
sys.path.append("./")
import os
import csv
import glob

from PIL import Image
import torch
from torchvision.transforms import transforms

from util import F, load_cosface, add_norm_to_model
from bim_attack import basic_iterative_method as BIM
from symmetry import symmetry_estimation


def __calculate_score_single__(model: torch.nn.Module, image_transform: transforms.Compose, image_path: str, eps: float, l: int, k: int, p: int) -> float:
    """Method performs FaceQAN over individual images

    Args:
        model (torch.nn.Module): FR model used for adversarial attack.
        image_transform (transforms.Compose): Image transformation, without normalization.
        image_path (str): Path to image, for which the quality score will be predicted.
        eps (float): ε-parameter from the paper.
        l (int): l-parameter from the paper, controling the number of FGSM iterations to perform.
        k (int): k-parameter from the paper, controling the number of adversarial examples generated.
        p (int): p-parameter from the paper, used in the final quality score calculation as the exponent.

    Returns:
        float: The calculated quality score
    """
    image = Image.open(image_path).convert("RGB")
    S_i = BIM(model, image_transform(image), eps=eps, iter=l)
    q_s = symmetry_estimation(model, image_transform, image)
    quality_score = F(S_i, q_s)
    return quality_score

eps,l,k,p = 0.001, 5, 10, 5
model = load_cosface().eval()
model = add_norm_to_model(model,mean=[.5, .5, .5],std=[.5, .5, .5])
image_transforms = transforms.Compose([transforms.Resize((112, 112)),transforms.ToTensor()])

for d,c in [(d,c) for d in range(1,4) for c in range(1,6)]:
  for (op, detector) in [(op, detector) for op in ["cropped","aligned"] for detector in ["mtcnn","rf"]]:
    scface = f'scface_d{d}_c{c}_{detector}_{op}'

    csv_file = f'/content/{scface}_faceqan.csv'
    dataset_path = f'/content/{scface}/*.jpg'

    print(scface)
    with open(csv_file, mode='w', newline='') as file:
      csv_writer = csv.writer(file)
      csv_writer.writerow(["Image","Score"])
      for index,image_path in enumerate(glob.glob(dataset_path)):
        qs = __calculate_score_single__(model,image_transforms,image_path,eps,l,k,p)
        csv_writer.writerow([image_path.split("/")[-1], qs])
        print(f'Image {index} -- {image_path} -- Quality score {qs}')


/content/FaceQAN/src/models/iresnet.py:207: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(os.path.dirname(__file__), "weights.p

scface_d1_c1_mtcnn_cropped


/content/FaceQAN/src/models/iresnet.py:154: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.fp16):


Streaming output truncated to the last 5000 lines.
Image 66 -- /content/scface_d2_c1_rf_cropped/068_cam1_2.jpg -- Quality score 0.1667798653433218
Image 67 -- /content/scface_d2_c1_rf_cropped/056_cam1_2.jpg -- Quality score 0.11308339005661547
Image 68 -- /content/scface_d2_c1_rf_cropped/034_cam1_2.jpg -- Quality score 0.3075677502245567
Image 69 -- /content/scface_d2_c1_rf_cropped/082_cam1_2.jpg -- Quality score 0.45324817712445264
Image 70 -- /content/scface_d2_c1_rf_cropped/042_cam1_2.jpg -- Quality score 0.5910730210305954
Image 71 -- /content/scface_d2_c1_rf_cropped/055_cam1_2.jpg -- Quality score 0.38631580186459463
Image 72 -- /content/scface_d2_c1_rf_cropped/081_cam1_2.jpg -- Quality score 0.6056102698959557
Image 73 -- /content/scface_d2_c1_rf_cropped/106_cam1_2.jpg -- Quality score 0.500574564697169
Image 74 -- /content/scface_d2_c1_rf_cropped/095_cam1_2.jpg -- Quality score 0.48213870512601925
Image 75 -- /content/scface_d2_c1_rf_cropped/001_cam1_2.jpg -- Quality score 0.565

In [ ]:
import shutil

for d,c in [(d,c) for d in range(1,4) for c in range(1,6)]:
  for (op, model) in [(op, model) for op in ["cropped","aligned"] for model in ["mtcnn","rf"]]:
    scface = f'scface_d{d}_c{c}_{model}_{op}'
    csv_file = f"/content/{scface}_faceqan.csv"
    source_file = csv_file
    destination_directory = "/content/drive/MyDrive/FaceAlignmentV2/SCFace"
    shutil.copy(source_file, destination_directory)
    print(f"File copied from {source_file} to {destination_directory}")

File copied from /content/scface_d1_c1_mtcnn_cropped_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c1_rf_cropped_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c1_mtcnn_aligned_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c1_rf_aligned_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_mtcnn_cropped_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_rf_cropped_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_mtcnn_aligned_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c2_rf_aligned_faceqan.csv to /content/drive/MyDrive/FaceAlignmentV2/SCFace
File copied from /content/scface_d1_c3_mtcnn_cropped_faceqan.csv to /content/drive/MyDrive/FaceAlign